In [1]:
import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm
import IPython.display as display
import copy
import seaborn as sns

/opt/anaconda3/envs/geospatial/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/1k/27mkp8bj3ps60c3nmr7rbqzh0000gn/T/ipykernel_14570/1895528008.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch shou

In [2]:
def merge_csv_files(directory):
    # Get a list of all the csv files
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

    # Initialize an empty list to hold dataframes
    dfs = []

    # Loop through csv files, read each into a dataframe, and append to the list
    for file in csv_files:
        # Extract date from filename, assuming the date is in format 'traffic_flow_YYYY_MM_DD'
        date_str = file.split('.')[0].split('_')[-3:]  # This gives ['YYYY', 'MM', 'DD']
        date = datetime.strptime('_'.join(date_str), '%Y_%m_%d').date()

        df = pd.read_csv(os.path.join(directory, file))

        # Add date as a new column
        df['date'] = date.strftime('%m/%d/%y')

        dfs.append(df)

    # Concatenate all dataframes in the list into one dataframe
    merged_df = pd.concat(dfs, ignore_index=True).drop_duplicates()

    # Return the merged dataframe
    return merged_df

In [3]:
traffic_flows = merge_csv_files(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]Traffic flow')
road_network = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]road_network/road_network.shp'

# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()

In [4]:
lsoa = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/London administrative boundaries/london_LSOA/london_LSOA.shp'
road_network = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]road_network/road_network.shp'
inoutter = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/London administrative boundaries/lp-consultation-oct-2009-inner-outer-london-shp/lp-consultation-oct-2009-inner-outer-london.shp'
tube_line = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_lines.json'
tube_station = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_stations.json'

inoutter = gpd.read_file(inoutter)
inoutter.to_crs(epsg=27700, inplace=True)

# tube_station = gpd.read_file(tube_station)
# tube_station.to_crs(epsg=27700, inplace=True)
# tube_station = gpd.sjoin(tube_station, inoutter, op='within')

tube_line = gpd.read_file(tube_line)
tube_line.to_crs(epsg=27700, inplace=True)
tube_line = gpd.sjoin(tube_line, inoutter, op='within')

lsoa = gpd.read_file(lsoa, crs={'init': 'epsg:27700'})
road_network = gpd.read_file(road_network, crs={'init': 'epsg:27700'})

/opt/anaconda3/envs/geospatial/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [5]:
# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()
traffic_flows['total'] = traffic_flows['bus'] + traffic_flows['car'] + traffic_flows['cycle'] + traffic_flows[
    'walks'] + traffic_flows['stationary']

In [16]:
flows = pd.merge(
    road_network[['toid', 'roadclassi', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg'
                  ]],
    traffic_flows, left_on='toid', right_on='toid', how='right')

flows['classification'] = flows['roadclassi'].replace(
    {'Unknown': 'Local Road', 'Not Classified': 'Local Road', 'Unclassified': 'Local Road',
     'Classified Unnumbered': 'Local Road'})

flows.drop(columns=['roadclassi'], inplace=True)

stage_date = ['03/01/22', '02/22/22', '03/08/22']
flows = flows.loc[flows['date'].isin(stage_date)]

# label the regional level
flows = gpd.sjoin(flows, inoutter, how='inner', predicate='within')
flows = flows.drop(columns=['index_right', 'Source', 'Area_Ha', 'Shape_Leng', 'Shape_Area'])

# convert the dataframe
flows = pd.melt(flows,
                id_vars=['toid', 'classification', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg',
                         'date', 'Boundary'], var_name='mode', value_name='flow')

flows = pd.pivot_table(flows,
                       index=['toid', 'classification', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg',
                              'date', 'Boundary', 'mode'], columns='date', values='flow', aggfunc='first').reset_index()

flows.drop(columns=['date'], inplace=True)
flows = flows.groupby(
    ['toid', 'mode', 'classification', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg',
     'Boundary'], as_index=False).agg(
    {'03/01/22': 'first', '02/22/22': 'first', '03/08/22': 'first'}).reset_index()

# calculate the impact and recovery flows for one strike
flows['impact_flow'] = flows['03/01/22'] - flows['02/22/22']
flows['recovery_flow'] = flows['03/08/22'] - flows['03/01/22']

flows = flows.drop(columns=['index'],inplace=True)


### K-means clustering

In [15]:
from sklearn.cluster import KMeans

# Group the DataFrame by unique values in 'mode' column
grouped = flows.groupby('mode')

# Perform KMeans clustering for each group separately
for mode, group in grouped:
    kmeans = KMeans(n_clusters=3, random_state=42)
    flows.loc[group.index, 'KmeansBymode_impact'] = kmeans.fit_predict(group[['impact_flow']])
    flows.loc[group.index, 'KmeansBymode_recovery'] = kmeans.fit_predict(group[['recovery_flow']])



In [ ]:
# Create a scatter plot to visualize 'KmeansBymode_impact' and its clustering results
plt.figure(figsize=(10, 6))
sns.scatterplot(data=flows, x='KmeansBymode_impact', y='impact_flow', hue='mode', palette='tab10', s=100)
plt.title('KMeans Clustering for Different Modes')
plt.xlabel('KmeansBymode_impact')
plt.ylabel('Impact Flow')
plt.legend(title='Mode', loc='upper right', bbox_to_anchor=(1.15, 1.0))
plt.show()


In [ ]:
def visualize_kmeans_by_mode_and_classification(flows, feature1, feature2):
    """
    Visualize KMeans clustering results for different 'mode' and 'classification' categories.

    Parameters:
        flows (DataFrame): The DataFrame containing the data.
        feature1 (str): The name of the first feature to visualize KMeans clusters.
        feature2 (str): The name of the second feature to visualize KMeans clusters.

    Returns:
        None
    """

    # Create a new column to store the combined KMeans cluster labels
    flows['KmeansByModeAndClassification'] = None

    # Group the DataFrame by unique values in 'mode' column
    grouped_by_mode = flows.groupby('mode')

    # Perform KMeans clustering for each 'mode' group separately
    for mode, group_mode in grouped_by_mode:
        kmeans = KMeans(n_clusters=3, random_state=42)
        flows.loc[group_mode.index, 'KmeansByModeAndClassification'] = kmeans.fit_predict(group_mode[[feature1, feature2]])

        # Create subplots for each 'classification' category
        plt.figure(figsize=(8, 6))
        sns.scatterplot(data=flows.loc[group_mode.index], x=feature1, y=feature2, hue='KmeansByModeAndClassification', palette='tab10', s=100)
        plt.title(f'KMeans Clustering for {mode} Mode')
        plt.xlabel(feature1)
        plt.ylabel(feature2)
        plt.legend(title='KMeans Cluster')
        plt.show()

# Call the visualization method
visualize_kmeans_by_mode_and_classification(flows, 'impact_flow', 'recovery_flow')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 假设你的DataFrame名为flows，包含'mode'、'toid'、'classification'和'impact_flow'等列

# 定义分类顺序
classification_order = ['Motorway', 'A Road', 'B Road', 'Local Road']
mode_order = ['total', 'car', 'bus', 'cycle', 'walks', 'stationary']

# 使用pivot_table将'mode'作为列，'classification'作为行，'impact_flow'作为值，并进行求和（sum）
flows_pivot = pd.pivot_table(flows, index='classification', columns='mode', values='impact_flow', aggfunc='sum')

# 对DataFrame进行排序，以调整顺序
flows_pivot = flows_pivot.reindex(classification_order, axis=0)
flows_pivot = flows_pivot[mode_order]

# 对impact_flow取对数
flows_pivot = np.log(flows_pivot)

# 绘制条形图
plt.figure(figsize=(10, 6))
flows_pivot.plot(kind='bar', ax=plt.gca())
plt.title('Impact Flow by Classification and Mode')
plt.xlabel('Classification')
plt.ylabel('Impact Flow')
plt.legend(title='Mode', loc='upper right', bbox_to_anchor=(1.15, 1.0))
plt.show()


In [ ]:
# 定义分类顺序和模式顺序
classification_order = ['Motorway', 'A Road', 'B Road', 'Local Road']
mode_order = ['total', 'car', 'bus', 'cycle', 'walks', 'stationary']

# 使用pivot_table将'mode'作为列，'classification'作为行，'impact_flow'作为值，并进行求和（sum）
flows_pivot = pd.pivot_table(flows, index='classification', columns='mode', values='impact_flow', aggfunc='sum')

# 对DataFrame进行排序，以调整顺序
flows_pivot = flows_pivot.reindex(classification_order, axis=0)
flows_pivot = flows_pivot[mode_order]

# 对impact_flow取对数
flows_pivot_log = np.log(flows_pivot)

# 根据'KmeansBymode_impact'列的值将flows DataFrame分为两部分
flows_impact_1 = flows[flows['KmeansBymode_impact'] == 1]
flows_impact_0 = flows[flows['KmeansBymode_impact'] == 0]

# 将flows DataFrame转换为geopandas GeoDataFrame
gdf_impact_1 = gpd.GeoDataFrame(flows_impact_1, geometry='geometry')
gdf_impact_0 = gpd.GeoDataFrame(flows_impact_0, geometry='geometry')

# 绘制空间可视化
fig, ax = plt.subplots(figsize=(10, 10))
gdf_impact_1.plot(ax=ax, color='red', legend=True, markersize='impact_flow', label='KmeansBymode_impact = 1')
gdf_impact_0.plot(ax=ax, color='blue', legend=True, markersize='impact_flow', label='KmeansBymode_impact = 0')
plt.title('Spatial Visualization of Flows')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(title='KmeansBymode_impact', loc='upper right', bbox_to_anchor=(1.15, 1.0))
plt.show()


### GNN


In [ ]:
import torch
from torch_geometric.data import Data
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.data import DataLoader
from torch.optim import Adam
from sklearn.cluster import KMeans

# Step 1: Create graph representation
# Create an adjacency matrix (assuming 'toid' is used as node IDs)
adjacency_matrix = pd.DataFrame(0, index=flows['toid'], columns=flows['toid'])
for _, row in flows.iterrows():
    adjacency_matrix.at[row['toid'], row['toid']] = 1

# Create impact_flow tensor (assuming 'impact_flow' is already prepared as a PyTorch tensor)
impact_flow_tensors = torch.tensor(flows['impact_flow'].values, dtype=torch.float)

# Convert other node attributes to PyTorch tensors
toid_tensors = torch.tensor(flows['toid'].values, dtype=torch.long)
mode_tensors = torch.tensor(flows['mode'].values, dtype=torch.long)
classification_tensors = torch.tensor(flows['classification'].values, dtype=torch.long)
geometry_tensors = torch.tensor(flows['geometry'].values, dtype=torch.float)
directiona_tensors = torch.tensor(flows['directiona'].values, dtype=torch.float)
length_tensors = torch.tensor(flows['length'].values, dtype=torch.float)
roadwidthm_tensors = torch.tensor(flows['roadwidthm'].values, dtype=torch.float)
elevationg_tensors = torch.tensor(flows['elevationg'].values, dtype=torch.float)
boundary_tensors = torch.tensor(flows['Boundary'].values, dtype=torch.long)

# Concatenate all node attributes into a single tensor for each node
x = torch.stack([impact_flow_tensors, toid_tensors, mode_tensors, classification_tensors, geometry_tensors,
                 directiona_tensors, length_tensors, roadwidthm_tensors, elevationg_tensors, boundary_tensors], dim=1)

# Create Data object for the graph
data = Data(x=x, edge_index=adjacency_matrix.values.nonzero())



# Step 3: Define the GNN model
class GNNModel(MessagePassing):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__(aggr='add')  # Use 'add' aggregation for message passing
        self.lin1 = nn.Linear(in_channels, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.lin1(x))
        x = self.propagate(edge_index, x=x)
        x = F.relu(self.lin2(x))
        return x

    def message(self, x_j):
        return x_j


# Step 4: Model training

# Prepare the DataLoader for training the GNN model
loader = DataLoader([data], batch_size=1)

# Create the GNN model instance
model = GNNModel(in_channels=data.num_node_features, hidden_channels=64, out_channels=32)

# Set optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.01)
loss_function = nn.MSELoss()

# Train the GNN model
model.train()
for data in loader:
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = loss_function(out, data.x)  # Assuming impact_flow is used as node feature
    loss.backward()
    optimizer.step()

# Step 5: Clustering and identifying roads with significant changes



# Assuming the model is already trained

# Get node embeddings from the trained model
model.eval()
with torch.no_grad():
    node_embeddings = model(data.x, data.edge_index)

# Use KMeans clustering on the node embeddings
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(node_embeddings.numpy())

# Add the cluster labels to the original DataFrame
flows['cluster'] = clusters

# Now, you can analyze the clusters and identify the roads with significant impact flow changes

